In [ ]:
from tensorflow import keras
import kagglehub

In [4]:
import pathlib
path = kagglehub.dataset_download("ayuraj/asl-dataset", download_path="C:\Users\user\Desktop\Sign Language Translator")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3333904223.py, line 2)

In [3]:
path

'C:\\Users\\user\\.cache\\kagglehub\\datasets\\ayuraj\\asl-dataset\\versions\\1'

In [1]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [3]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [5]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,
)
datagen_valid = ImageDataGenerator(samplewise_center=True)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [7]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "data/fruits/train/",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "data/fruits/valid/",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting the desired number of epochs.

In [8]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=9)

Epoch 1/9
37/36 [==============================] - 26s 694ms/step - loss: 2.5725 - accuracy: 0.4239 - val_loss: 1.0127 - val_accuracy: 0.6991
Epoch 2/9
37/36 [==============================] - 17s 470ms/step - loss: 0.7845 - accuracy: 0.7352 - val_loss: 0.4406 - val_accuracy: 0.8693
Epoch 3/9
37/36 [==============================] - 17s 466ms/step - loss: 0.4457 - accuracy: 0.8418 - val_loss: 0.3310 - val_accuracy: 0.8875
Epoch 4/9
37/36 [==============================] - 17s 457ms/step - loss: 0.2950 - accuracy: 0.8942 - val_loss: 0.2288 - val_accuracy: 0.9301
Epoch 5/9
37/36 [==============================] - 17s 457ms/step - loss: 0.2355 - accuracy: 0.9146 - val_loss: 0.1970 - val_accuracy: 0.9392
Epoch 6/9
37/36 [==============================] - 17s 462ms/step - loss: 0.1811 - accuracy: 0.9340 - val_loss: 0.2193 - val_accuracy: 0.9149
Epoch 7/9
37/36 [==============================] - 17s 464ms/step - loss: 0.1280 - accuracy: 0.9442 - val_loss: 0.1507 - val_accuracy: 0.9453
Epoch 

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [16]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10 )

Epoch 1/10
37/36 [==============================] - 31s 827ms/step - loss: 0.1083 - accuracy: 0.9611 - val_loss: 0.1446 - val_accuracy: 0.9666
Epoch 2/10
37/36 [==============================] - 19s 507ms/step - loss: 0.0825 - accuracy: 0.9695 - val_loss: 0.1075 - val_accuracy: 0.9726
Epoch 3/10
37/36 [==============================] - 18s 477ms/step - loss: 0.0472 - accuracy: 0.9873 - val_loss: 0.1595 - val_accuracy: 0.9666
Epoch 4/10
37/36 [==============================] - 18s 485ms/step - loss: 0.0353 - accuracy: 0.9873 - val_loss: 0.1698 - val_accuracy: 0.9666
Epoch 5/10
37/36 [==============================] - 18s 487ms/step - loss: 0.0529 - accuracy: 0.9822 - val_loss: 0.0960 - val_accuracy: 0.9757
Epoch 6/10
37/36 [==============================] - 19s 504ms/step - loss: 0.0216 - accuracy: 0.9932 - val_loss: 0.1878 - val_accuracy: 0.9696
Epoch 7/10
37/36 [==============================] - 18s 482ms/step - loss: 0.0274 - accuracy: 0.9924 - val_loss: 0.1136 - val_accuracy: 0.9666

## Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)